<table>
    <tr>
        <td>
            <img src="extra/logo_scgis.png" width="100" height="100" />
        </td>
        <td>
            <center>
                <font size = 1>
                8 НОЯБРЯ 2020<br>
                АНТИКОНФЕРЕНЦИЯ СООБЩЕСТВА ПРИРОДООХРАННЫХ ГИС В РОССИИ<br><br>
                </font>
                <font size = 3>
                Материалы к вебинару<br> <b>Получение и обработка данных дистанционного зондирования с помощью Python:<br>строим простую систему мониторинга окружающей среды в Jupyter Notebook.</b><br><br>
                </font>
                <font size = 2>
                <b>Эдуард Казаков</b><br>
                NextGIS, Государственный гидрологический институт
                </font>
            </center>
        </td>
    </tr>
</table>

# Преамбула

Эти материалы призваны познакомить интересующихся с некоторыми аспектами использования языка Python для решения задач, связанных с организацией мониторинга окружающей среды на базе открытых данных дистанционнго зондирования Земли из космоса. Jupyter - удобная среда для демонстрации основных идей, но есть множество других способов использовать приведенный код.

Несмотря на появление множества потрясающих инструментов, таких как Google Earth Engine, умение запрограммировать собственную цепочку обработки остаётся очень ценным навыком. У внешних платформ всегда будут ограничения по доступным данным и набору инструментов обработки, а в своём собственном коде вы ограничены только временем своей жизни на разработку :)

Рассматриваемая задача достаточно условна, но включает работу с распространенными данными и способами взаимодействия с ними.

&#128151; Подготовлено специально для прекрасных людей, связанных с сообществом природоохранных ГИС или сочувствующих ему, в 2020 году. 

# Постановка задачи
Мониторинг - фундаментальный и понятийно очень простой процесс, лежащий в основе взаимодействия человека и окружающего мира. Говоря простым языком, под мониторингом можно понимать любое периодическое наблюдение за состоянием отдельного природного объекта или природного комплекса. Например, если вы каждое утро спускаетесь к реке и по некоторой шкале оцениваете её мутность, ведя историю таких замеров, с чистой совестью можно всем вокруг рассказывать, что вы осуществляете мониторинг окружающей среды - и это будет правдой.

Наступление эпохи дистанционного зондирования открыло огромные мониторинговые возможности, ведь теперь за планетой непрерывно наблюдают сотни космических аппаратов. В зависимости от используемых источников, каждый месяц, неделю, день, а иногда и чаще, мы получаем цифровой отпечаток состояния любого уголка Земли. Накопленные за вот уже полувековую историю космических наблюдений данные перевернули наше представления о родной планете, а главное - каждую секунду эти знания продолжают пополняться.

Благодаря открытым программам ДЗЗ (<a href="https://www.copernicus.eu/en">ESA Copernicus</a>, <a href="https://eospso.nasa.gov/content/nasas-earth-observing-system-project-science-office">NASA EOS</a> и многие другие) бесплатный доступ к постоянно обновляемым архивам космической съемки имеет всё человечество (подключенное к сети Интернет). А это значит, что даже самые частные задачки, связанные с периодическим наблюдением за состоянием поверхности, могут быть хотя бы в первом приближении решаемы любым человеком в любой части мира, будь то исполненный надежд школьник или суровый лесохозяйственник.

Мы попробуем сделать простую и минималистичную "систему мониторинга", которая при каждом запуске будет получать свежие данные ДЗЗ на выбранную территорию, немного их обрабатывать и представлять результаты в виде человекочитаемых сводок: статистических, графических и картографических. 

## Данные

В качестве источников данных выберем два:
* <a href="https://sentinel.esa.int/web/sentinel/missions/sentinel-2">Sentinel 2</a> - наиболее популярные сегодня данные с относительно высоким пространственным разрешением. Приблизительно каждые 5 суток на любую часть планеты мы получаем такие данные с детальностью около 10 метров на пиксель. Это очень хорошее качество. 
* <a href="https://modis.gsfc.nasa.gov/about/">MODIS</a> - Данные MODIS позволяют получать ежедневные данные с разрешением от 250 метров на пиксель. Несмотря на вроде бы низкое разрешение (на самом деле нет!), эти данные незаменимы для мониторинга динамических масштабных процессов, таких как лесные пожары, крупные наводнения, дрейф морских льдов и так далее.

Выбор также обусловлен методическими целями. Дело в том, что способ получения у этих двух источников данных весьма различен. Для получения данных Sentinel мы воспользуемся программным интерфейсом (API), то есть будем формировать специальные запросы к официальному шлюзу, который в ответ будет предлагать нам загрузить подходящие данные. А для получения данных MODIS мы будем идти в специальное файловое хранилище, где по имени файла сами будем разбираться, что нужно загрузить.

## Задача

Мы хотим следить за динамикой значений вегетационного индекса <a href="https://gis-lab.info/qa/ndvi.html">NDVI</a> в пределах некоторого участка территории. Участок территории определяется содержимым векторного слоя.

При каждом запуске должны догружаться новые данные, маскироваться облачность, рассчитываться NDVI, извлекаться данные в пределах участка.

Для каждого участка на даты наличия космических снимков мы хотим видеть отчёт:
* Среднее, минимальное и максимальное значения NDVI
* График динамики NDVI за всю накопленную историю наблюдений
* Карту распределения для последних данных поверх картографической подложки

### Импортируем библиотеки, возможности которых нам будут необходимы

In [3]:
# Инструменты для работы с файловой системой
import os

# Библиотеки для работы с векторными и растровыми данными
import gdal, ogr

# Библиотека для взаимодействия с интернет-ресурсами
import requests

In [4]:
# TODO: ...
